# Zhang's Algorithm For Camera Calibration

### Import Statements

In [1]:
import os
from camera_callibration_helper import *
import cv2
import numpy as np
from copy import deepcopy
from tqdm import tqdm

### Load the Images
* raw_img_list (list): list of 40 BGR input images
* grey_img_list (list): list of 40 grey scale input images
* img_labels (list): list of 40 image filenames (mainly for debugging)

In [2]:
given_data_path = "/home/jo_wang/Desktop/ECE661/HW08/Dataset1"
raw_img_list, grey_img_list, img_labels = loadImages(given_data_path)
assert(len(grey_img_list) == 40)
assert(len(raw_img_list) == 40)
assert(len(img_labels) == 40)

### Apply Canny Edge Detector On Grey Scale Images
* edge_img_list (list): list of edge maps from Canny

In [3]:
edge_img_list = performCanny(grey_img_list)
assert(len(edge_img_list) == 40)

In [4]:
hough_lines_list = performHoughTransform(edge_img_list)
assert(len(hough_lines_list) == len(edge_img_list))

In [7]:
for i in tqdm(range(40)):
    i = 4

    h_lines, v_lines = get_Horizontal_Vert_Lines(hough_lines_list[i])

    v_lines = np.array(v_lines).reshape(-1,2)
    h_lines = np.array(h_lines).reshape(-1,2)

    corner_points = getCorners(v_lines, h_lines)
    print(corner_points)
    img = deepcopy(raw_img_list[i])
    for i, point in enumerate(corner_points):
        try:
            img = cv2.circle(img, point, 3, (0, 0, 255), -1)
            cv2.putText(img, str(i), (point[0]+5, point[1]-5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1)
        except OverflowError:
            pass
    cv2.imshow('test',img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()







  0%|          | 0/40 [00:00<?, ?it/s]

([[2.0, 1.5620697], [-52.0, 2.0769417], [159.33333, 1.0471975], [2.0, 1.5620697], [144.66667, 1.0471975], [1.5, 1.5620697], [2.5, 1.5620697], [1.5, 1.5620697]], [[193.0, 1.5620697], [400.5, 1.579523], [68.0, 1.553343], [279.5, 1.5620697], [150.0, 1.5620697], [361.66666, 1.5707964], [441.5, 1.579523], [236.5, 1.5620697], [107.5, 1.5620697], [319.0, 1.5707964]])


SystemError: new style getargs format but argument is not a tuple

In [15]:
print(img_labels[4])
h_lines, v_lines = get_Horizontal_Vert_Lines(hough_lines_list[4])

v_lines = np.array(v_lines).reshape(-1,2)
h_lines = np.array(h_lines).reshape(-1,2)

v_lines_sorted, h_lines_sorted = getCorners(v_lines, h_lines)

print(v_lines_sorted)
print(v_lines)
for l in h_lines_sorted:
    rho = l[0]
    theta = l[1]
    L = 1000 #np.sqrt(m**2 + n**2)
    a = np.cos(theta)
    b = np.sin(theta)
    x0 = a*rho
    y0 = b*rho
    x1 = int(x0 + L*(-b))
    y1 = int(y0 + L*(a))
    x2 = int(x0 - L*(-b))
    y2 = int(y0 - L*(a))
    # print((x1,y1),(x2,y2))
    cv2.line(raw_img_list[4],(x1,y1),(x2,y2),(0,0,255),2)
cv2.imshow("test", raw_img_list[4])
cv2.waitKey(0)
cv2.destroyAllWindows()

Pic_13.jpg
[[2.0, 1.5620697], [-52.0, 2.0769417], [159.33333, 1.0471975], [2.0, 1.5620697], [144.66667, 1.0471975], [1.5, 1.5620697], [2.5, 1.5620697], [1.5, 1.5620697]]
[[-2.0700000e+02  3.1241393e+00]
 [ 3.8200000e+02  0.0000000e+00]
 [-1.6500000e+02  3.1241393e+00]
 [-2.4900000e+02  3.1241393e+00]
 [-2.9100000e+02  3.1241393e+00]
 [ 2.1000000e+02  0.0000000e+00]
 [ 4.2400000e+02  0.0000000e+00]
 [-4.2100000e+02  3.1241393e+00]
 [-3.3500000e+02  3.1241393e+00]
 [ 4.6800000e+02  0.0000000e+00]
 [ 1.6800000e+02  0.0000000e+00]
 [ 2.9500000e+02  0.0000000e+00]
 [-3.7800000e+02  3.1241393e+00]
 [-4.6400000e+02  3.1241393e+00]
 [ 3.4000000e+02  0.0000000e+00]
 [ 4.3100000e+02  1.7453292e-02]
 [ 4.7400000e+02  1.7453292e-02]
 [ 2.5200000e+02  0.0000000e+00]
 [-1.5900000e+02  3.1066861e+00]]


In [ ]:
world_points = list()
for i in range(0, 200, 20):
    for j in range(0, 160, 20):
        world_points.append([i,j])
assert(len(world_points) == 80)